# Blockchain Explained in 7 Python Functions
https://www.kdnuggets.com/2018/04/blockchain-explained-7-python-functions.html

This notebook will go over the kdnuggest post in an attempt to better understand blockchain. It is nearly a verbatium copy and paste of the link above (with some small smattering of my own thoughts).

# Hash Function
At the heart of the blockchain is the hashing function. Without encryption, the blockchain will be easily manipulated and transactions will be able to be fraudulently inserted.

So it seems that blockchain is all about transactions.

In [15]:
import json
import hashlib

def hash_function(k):
    """Hashes our transaction"""
    if type(k) is not str:
        k = json.dumps(k, sort_keys=True)
    
    return hashlib.sha256(k.encode('utf-8')).hexdigest()

# I tried running this with a simple stirng "test" and it failed.

# Update the state of the blockchain
The state of the blockchain is the record of who owns what. It should be noted that overdrafts cannot exist. This means that there must also be a function that checks for the validity of the transaction before allowing the transaction to occur.

In [9]:
def update_state(transaction, state):
    state = state.copy()
    
    for key in transaction:
        if key in state.keys():
            state[key] += transaction[key]
        else:
            state[key] = transaction[key]
    
    return state


def valid_transaction(transaction, state):
    """A valid transaction must sum to 0"""
    
    if sum(transaction.values()) is not 0:
        return False
    
    for key in transaction.keys():
        if key in state.keys():
            account_balance = state[key]
        else:
            account_balance = 0
        
        if account_balance + transaction[key] < 0:
            return False
    
    return True


# Make the block
Next we make the block. In general, information from the previous block is read, and used to link it to the new block. This too is central to the idea of blockchain. Seemingly valid transactions can be attempted to fradulently be inserted into the blockchain, but decrypting all the previous blocks is computationally (nearly) impossible. This preserves the integrity of the blockchain.

In [10]:
def make_block(transactions, chain):
    """Make a block to go into the chain."""
    parent_hash = chain[-1]['hash']
    block_number = chain[-1]['contents']['block_number'] + 1
    
    block_contents = {
        'block_number': block_number,
        'parent_hash': parent_hash,
        'transaction_count': block_number + 1,
        'transaction': transactions
    }
    
    return {'hash': hash_function(block_contents), 'contents': block_contents}

def check_block_hash(block):
    expected_hash = hash_function(block['contents'])
    
    if block['hash'] is not expected_hash:
        raise
    return

# Updating the blockchain

In [11]:
def check_block_validity(block, parent, state):
    parent_number = parent['contents']['block_number']
    parent_hash = parent['hash']
    block_number = block['contents']['block_number']
    
    for transaction in block['contents']['transaction']:
        if valid_transaction(transaction, state):
            state = update_state(transaction, state)
        else:
            raise
            
    check_block_hash(block)
    
    if block_number is not parent_number + 1:
        raise
    
    if block['contents']['parent_hash'] is not parent_hash:
        raise
        
    return state        

# Verify the chain

In [12]:
def check_chain(chain):
    """Check to see that the chain is valid."""
    if type(chain) is str:
        try:
            chain = json.loads(chain)
            assert (type(chain) == list)
        except ValueError:
            #string passed was not JSON
            return False
    elif type(chain) is not list:
        return False
    
    state = {}
    
    for transaction in chain[0]['contents']['transaction']:
        state = update_state(transaction, state)
    
    check_block_hash(chain[0])
    parent = chain[0]
    
    for block in chain[1:]:
        state = check_block_validity(block, parent, state)
        parent = block
        
    return state

# Transaction Function -- Pulling everything together

In [13]:
def add_transaction_to_chain(transaction, state, chain):
    if valid_transaction(transaction, state):
        state = update_state(transaction, state)
    else:
        raise Exception('Invalid Transaction.')
        
    my_block = make_block(state, chain)
    chain.append(my_block)
    
    for transaction in chain:
        check_chain(transaction)
        
    return state, chain

# Cool now what?
The functions have been made, so how do we do anything with it? First, we need to make a Gensis block. This is the inception of a new coin (or whatever). For the purposes of this example, I (and I am plagerizing) I will start of with 10 coins for Jasper

In [16]:
gensis_block = {
    'hash': hash_function({
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Jasper':10}]
    }),
    'contents': {
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Jasper': 10}]
    }
}

block_chain = [gensis_block]
chain_state = {'Jasper': 10}

# Lets make a transaction!
Jasper will give Kyle 2 coins

In [17]:
chain_state, block_chain = add_transaction_to_chain(transaction={"Jasper": -2, 'Kyle':2}, state=chain_state, chain=block_chain)

In [22]:
print(chain_state)
block_chain

{'Jasper': 8, 'Kyle': 2}


[{'contents': {'block_number': 0,
   'parent_hash': None,
   'transaction': [{'Jasper': 10}],
   'transaction_count': 1},
  'hash': 'cab6513a8f1144306c3576ad1f159a40ed127531011c73332f3d93b2785b4293'},
 {'contents': {'block_number': 1,
   'parent_hash': 'cab6513a8f1144306c3576ad1f159a40ed127531011c73332f3d93b2785b4293',
   'transaction': {'Jasper': 8, 'Kyle': 2},
   'transaction_count': 2},
  'hash': '07b2354965c86e1123cc01a9ba120e5be9eea437fd96a5af84d123a54a057c08'}]